In [2]:
%matplotlib
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
path = os.path.join(os.getcwd(), 'driftData')
labels = ['copper grid',]

suffix = '_DRIFT.TXT'
timeStrings = [
        '20151130-122822',
        '20151130-123210',
        '20151130-124456',
        '20151130-125003',
        '20151130-131542',
        '20151130-135715',
        '20151130-142210',
        '20151130-145615',
        '20151130-152105',
        '20151130-155659',]

fileNames = [t + suffix for t in timeStrings]
times = [time.mktime(time.strptime(t, '%Y%m%d-%H%M%S')) for t in timeStrings]
colLabels = ['t','y','x','z']
colColours = [None, 'g', 'r', 'b']
skipRows = 1
skipCols = 1
fitPts = 4
temperatureFileName = '20151130-Temperatures.txt'
temperatureLabels = ['dewar', 'chamber', 'bridge']

Using matplotlib backend: TkAgg


In [5]:
data = []
for fn in fileNames:
    fp = os.path.join(path, fn)
    d = np.genfromtxt(fp, skip_header=skipRows)[:,skipCols:]
    # Substract baselines.
    for col in range(1,d.shape[1]):
        d[:,col] -= d[:,col].min()
    data.append(d)

temperatures = np.genfromtxt(os.path.join(path, temperatureFileName))[:,:]


In [6]:
aligned = np.concatenate([d for d in data])
tOffsets = [228, 994, 1304, 2840, 5340]
for i in range(len(data)):
    if i == 0:
        count = data[i].shape[0]
        continue
    numPoints = data[i].shape[0]
    # Manual t offset.
    aligned[count:count+numPoints,0] += times[i] - times[0]
    # Gradient
    deltaT = aligned[count-1, 0] - aligned[count, 0]
    for v in range(1,4):
        # Fit a stright line through a few points to estimate dvdt.
        dvdt = np.polyfit(aligned[count-fitPts-1:count-1, 0],
                      aligned[count-fitPts-1:count-1, v],
                      1)[0]   
        dv =  aligned[count,v]  + dvdt * deltaT - aligned[count-1, v]
        aligned[count:, v] -= dv
    count += numPoints   

# Gradient by fitting in microns per second.
gradient = np.zeros(aligned.shape)
for col in [1,2,3]:
    for i in range(2, aligned.shape[0]-2):
        gradient[i, col] = np.polyfit(aligned[i-2:i+2, 0],
                                 aligned[i-2:i+2, col],
                                 1)[0]
# Convert to nm / second
gradient *= 1000.

# Subtract start time from temperature-log time points.
startTime = 1448886502.


temperatures[:,0] = np.divide(np.add(temperatures[:,0], -times[0]), 60.)


In [8]:
fig = plt.figure()
fig.suptitle('Drift under cryo conditions')
gs = gridspec.GridSpec(3, 1, height_ratios=[1, 3, 1]) 

# Temperature
axTemp = fig.add_subplot(gs[2])
axTemp.xaxis.label.set_text('time / minutes')
axTemp.yaxis.label.set_text(u"temp. / °C")
axTemp.plot(temperatures[:,0], temperatures[:,1:])
axTemp.legend(temperatureLabels, loc='upper left', prop={'size':11})

# Position
axPos = fig.add_subplot(gs[1], sharex=axTemp)
axPos.yaxis.label.set_text(u'displacement / µm')

# Drift rate
axDrift = fig.add_subplot(gs[0], sharex=axTemp)
axDrift.yaxis.label.set_text(r'drift / nm.s$^-1$')

handles = []
for col in [3,1,2]:
    handles.append(
            axPos.plot(aligned.T[0]/60, aligned.T[col], label=colLabels[col],
                 ls=' ', marker='+', color=colColours[col]))
    axDrift.plot(aligned[:gradient.shape[0],0]/60, gradient[:,col],
             ls=' ', marker='.', markersize=2, color=colColours[col])

axPos.legend(*axPos.get_legend_handles_labels(), loc='upper left', prop={'size':11})

# Set axis limits
tMax = 210
axTemp.set_xlim(0, tMax)
axTemp.set_xticks(range(0, int(tMax+1), 15))
axDrift.set_ylim(-25, 25)

# Show grid lines on all axes.
for ax in (axPos, axDrift, axTemp):
    ax.grid(b=True, which='major', alpha=0.2, linestyle='-')

# Hide x-labels on all but one axis.
for ax in (axDrift, axPos):
    plt.setp(ax.get_xticklabels(), visible=False)